# 다중 회귀 분석
앞의 단순선형회귀에 추가적인 데이터를 사용해서 모델의 성능을 높여보자. \
시간(분) = $\alpha$ + ${\beta}_1 친구수$ + ${\beta}_2 근무시간$ + ${\beta}_3 박사 학위 취득 여부$ + $\epsilon$ \
박사 학위 취득 여부는 숫자 데이터가 아니다. 하지만 앞서 11장 '기계학습'에서 다룬 바와 같이 **가변수(dummy variable)** 를 만들어서 박사 학위를 가진 사람을 1, 그렇지 않은 사람을 0으로 표기하면 숫자로 표기할 수 있다.

## 15.1 모델
14장. ' 단순 회귀 분석' 에서는 아래와 같은 형태의 모델을 다뤘다. \
$y_i = \alpha + \beta x_i + \epsilon_i$ \
각 입력값 $x_i$는 숫자 하나가 아니라 k개의 숫자인 $x_i1, ..., x_ik$ 라고 한다면 다중 회귀(multiple regression)모델은 다음과 같은 형태를 띈다. \
$y_i = \alpha + \beta_1 x_i + ... + \beta_k x_ik + \epsilon_i$ \
다중 회귀 분석에서는 보통 파라미터 벡터를 $\beta$라고 부른다. 여기에 상수항까지 포함시키기 위해 데이터의 앞부분에 1로 구성된 열을 덧붙이면 된다. \
beta = [alpha, beta_1, ..., beta_k] \
그리고 각 데이터는 다음과 같다. \
x_i = [1, x_i1, ..., x_ik] \
이렇게 하면 모델을 다음과 같이 나타낼 수 있다.

In [1]:
from typing import Tuple, List
import math

Vector = List[float]

def dot(v: Vector, w: Vector) -> float:
    """v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w),  "vectors must be same length"
    
    return sum(v_i * w_i for v_i, w_i in zip(v,w))

In [2]:
def predict(x: Vector, beta: Vector) -> float:
    """각 x_i의 첫 번째 항목은 1이라고 가정"""
    return dot(x, beta)

In [3]:
# 이 경우 독립 변수 x는 각각 다음과 같은 벡터들의 열로 표현할 수 있다.
[1, # 상수항
49, # 친구의 수
4, # 하루 근무 시간
0] # 박사 학위 없음


[1, 49, 4, 0]

## 15.2 최소자승법에 대한 몇 가지 추가 가정
이 모델(그리고 답)이 의미가 있으려면 몇 가지 추가적인 가정이 필요하다. \
첫번째로 x의 열은 서로 **선형독립(linear independence)** 해야 한다. \
선형독립이란? \
어떤 벡터도 다른 벡터의 선형결합으로 만들어질 수 없다는 것을 의미한다. 만약 이 가정이 성립하지 않는다면 베타를 추정할 수 없다. 극단적인 예로 num_friends와 동일한 num_acquaintances가 데이터에 추가되었다고 해보자. \
그렇다면 어떠한 beta를 사용해도 num_friends 계수에 임의의 값을 더하고 num_acquaintances 계수에서 똑같은 값을 빼주면 모델의 예측값은 변하지 않을 것이다. 즉, num_friends의 정확한 계수를 계산할 수 없다는 것의 의미한다.(보통은 이 가정이 위배되었는지 확인하는 것은 쉽지 않다.) \
두번째로 중요한 가정은 $x$의 모든 열의 오류 $\epsilon$과 아무런 상관 관계가 없다는 것이다. 만약 이 가정이 위배되는 경우 아예 잘못된 beta가 추정될 것이다. \
예를 들어, 14장 '단순 회귀 분석'에서 만든 모델을 살펴보면 친구 수 가 한 명씩 증가할 때 사용자가 하루 평균 사이트에서 보내는 시간이 0.90분씩 증가한다고 예측되었다. 또한 다음과 같은 경우가 있다고 생각해 보자. 
- 근무 시간이 더 긴 사람은 더 적은 시간을 사이트에서 보낼 것이다.
- 친구 수가 많은 사람일수록 근무 시간이 더 길다.

이 경우 '실제' 모델은 다음과 같다.\
사이트에서 보내는 시간(분) = $\alpha$  + $\beta_1 친구수$ + $\beta_2 근무 시간$ + $\epsilon$ 

만약 실제 모델의 $\beta_1$(즉, '실제' 모델의 오류를 최소화하는 $\beta_1$)을 사용하는 단일 변수 모델로 예측 성능을 평가해 보자. $\beta_2$ < 0 이지만 모델에 포함시키지 않았기 때문에 근무 시간이 긴 사용자의 예측값은 너무 크게 계산될 것이고, 그눔 시간이 짧은 사용자의 예측값은 조금 더 크게 계산될 것이다. 또한 근무 시간과 친구의 수는 양의 상관관계를 지니기 때문에 친구 수가 많은 사용자의 예측값은 너무 크게 계산될 것이고, 친구 수가 적은 사용자의 예측값은 조금만 크게 계산될 것이다. \
결국 단일 변수 모델의 오류를 줄이기 위해서는 추정된 $\beta_1$을 줄여야 한다. 즉, 오류를 최소화하는 $\beta_1$은 실제 값보다 작아진다는 것을 의미한다. 결국 단일 변수 모델 안의 최소자승법의 결과는 $\beta_1$을 과소평가하도록 편향된다. 일반적으로 이렇게 독립 변수와 오류 사이에 상관관계가 존재한다면, 최소자승법으로 만들어지는 모델을 $\beta_1$를 추정해 준다.


## 15.3 모델 학습하기
단순 회귀 분석 모델처럼 오류를 제곱한 값의 합을 최소화해 주는 beta를 찾을 것이다. 경사 하강법을 사용하자. 오류의 제곱 합을 최소화할 것이며 이 경우 오류 함수는 14장 '단순 회귀 분석'에서 사용한 것과 같지만 [alpha, beta]를 받는 대신 임의의 벡터를 받도록 수정할 것이다.

In [7]:
from typing import List

def error(x: Vector, y: float, beta: Vector) -> float:
    return predict(x, beta) - y

def squared_error(x: Vector, y: float, beta: Vector) -> float:
    return error(x, y, beta) ** 2

In [9]:
x = [1,2,3]
y = 30
beta = [4,4,4]  # 예측 결과 = 4 + 8 + 12 = 24

In [10]:
assert error(x, y, beta) == -6
assert squared_error(x, y, beta) == 36

임의의 데이터를 다룰 수 있는 least-squares_fit 함수를 작성하자. 

In [12]:
import random
import tqdm

def scalar_multiply(c: float, v: Vector) -> Vector:
    """모든 성분을 c로 곱하기"""
    return [c * v_i for v_i in v]

def vector_sum(vectors: List[Vector]) -> Vector:
    """모든 벡터의 각 성분들끼리 더한다."""
    # vectors가 비어있는지 확인
    assert vectors, "no vectors provided"
    
    # 모든 벡터의 길이가 동일한지 확인
    num_elements = len(vectors[0])
    assert all(len(v) == num_elements for v in vectors), "different sizes!"
    
    # i번째 결과값은 모든 벡터의 i번째 성분을 더한 값
    return [sum(vector[i] for vector in vectors)
            for i in range(num_elements)]

def vector_mean(vectors: List[Vector]) -> Vector:
    """각 성분별 평균을 계산"""
    n = len(vectors)
    return scalar_multiply(1/n, vector_sum(vectors))

def gradient_step(v: Vector, gradient: Vector, step_size: float) -> Vector:
    """v에서 step_size만큼 이동하기"""
    assert len(v) == len(gradient)
    step = scalar_multiply(step_size, gradient)
    return add(v, step)

In [15]:
def least_squares_fit(xs: List[Vector],
                     ys: List[float],
                     learning_rate: float = 0.001,
                     num_steps: int = 1000,
                     batch_size: int = 1) -> Vector:
    """
    오류의 제곱 합을 최소화하는 beta를 찾자.
    모델은 y = dot(x, beta)라 가정하자.
    """
    # 임의의 위치에서 출발
    guess = [random.random() for _ in xs[0]]
    for _ in tqdm.trange(num_steps, desc="least squares fit"):
        for start in range(0, len(xs), batch_size):
            batch_xs = xs[start: start+batch_size]
            batch_ys = ys[start: start+batch_size]
            
            gradient = vector_mean([sqerror_gradient(x, y, guess)
                                   for x, y in zip(batch_xs, batch_ys)])
            guess = gradient_step(guess, gradient, -learning_rate)
    
    return guess

데이터에 적용

In [16]:
# 5장. 통계 import daily_minutes_good
def gradient_step(v: Vector, gradient: Vector, step_size: float) -> Vector:
    """v에서 step_size만큼 이동하기"""
    assert len(v) == len(gradient)
    step = scalar_multiply(step_size, gradient)
    return add(v, step)

In [17]:
random.seed(0)
# 시행착오를 통해 num_iters와 step_size를 선택하였다.
# 실행하는 데 시간이 좀 걸릴 것이다.
learning_rate = 0.001

beta = least_squares_fit(inputs, daily_minutes_good, learning_rate, 5000, 25)
assert 30.50 < beta[0] < 30.70  # 상수
assert 0.96 < beta[1] < 1.00  # 친구 수
assert -1.89 < beta[2] < -1.85  # 일 업무 시간
assert 0.91 < beta[3] < 0.94  # 박사 학위 여부


NameError: name 'inputs' is not defined

실제로는 경사 하강법으로 선형 회귀 모델을 추정하지 않는다. 선형대수 기법을 사용하면 정확한 계수를 구할 수 있지만 이 책에서 다룰 내용은 아니다. 만약 선형대수 방식을 사용했다면 다음과 같은 수식을 찾았을 것이다. \
사이트에서 보낸 시간(분) = 30.58 + 0.972 친구 수 - 1.87 근무 시간 + 0.923 박사 학위 취득 여부

## 15.4 모델 해석하기

모델의 계수는 다른 모든 것이 동일할 때 해당 항목의 영향력을 나타낸다. \
다른 모든 것이 동일할 때 친구 수가 한 명 증가하면 사용자가 하루 평균 사이트에서 보내는 시간은 1분 증가한다. \
다른 모든 것이 동일할 때 근무 시간이 한 시간 증가하면 사용자가 하루 평균 사이트에서 보내는 시간은 대략 2분 감소한다. 다른 모든 것이 동이할 때 박사 학위를
취득했다면 사용자는 하루 평균 사이트를 1분 더 사용한다. \
위의 해석은 변수 간의 관계를 직접적으로 설명해 주지 못한다. \

e.g) \
친구의 수가 많은 사용자의 근무 시간과, 적은 사용자들의 근무 시간은 서로 다른 방식으로 동작할 수도 있다. 이 모델은 이러한 관계를 잡아내지 못한다. \
이러한 문제를 다루는 방법 중 하나는 친구 수와 근무 시간을 곱한 새로운 변수를 도입하는 것이다. 이를 통해 친구의 수가 증가할 수록 근무 시간의 계수를 증가(혹은 감소)시킬 수 있다. \
혹은 친구 수가 증가할수록 사이트에서 보내는 시간은 어느 일정한 수준까지 증가하고, 그 이후로는 사이트에서 보내는 시간이 감소할 수도 있다. 이러한 현상은 친구수를 제곱한 값을 모델의 변수로 사용해서 잡아낼 수 있다. \
변수가 점점 추가되기 시작하면 각 계수가 유의미한지 살펴봐야 한다. 변수끼리 곱한 값, 변수의 log값, 변수의 제곱 값 등 수많은 새로운 변수를 추가할 수 있기 때문이다.

## 15.5 적합성(Goodness of fit)


In [19]:
from typing import List

def mean(xs: List[float]) -> float:
    return sum(xs) / len(xs)

def de_mean(xs: List[float]) -> List[float]:
    """x의 모든 데이터 포인트에서 평균을 뺌(평균을 0으로 만들기 위해)"""
    x_bar = mean(xs)
    return [x - x_bar for x in xs]

def total_sum_of_squares(y: Vector) -> float:
    """평균을 기준으로 y_i의 변화량을 제곱한 값의 총합"""
    return sum(v ** 2 for v in de_mean())

In [20]:
def multiple_r_squared(xs: List[Vector], ys: Vector, beta: Vector) -> float:
    sum_of_squared_errors = sum(error(x, y, beta) ** 2 for x, y in zip(xs, ys))
    return 1.0 - sum_of_squared_errors / total_sum_of_squares(ys)

In [21]:
assert 0.67 < multiple_r_squared(inputs, daily_minutes_good, beta) < 0.68

NameError: name 'inputs' is not defined